In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_timestamp, collect_set, struct
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType
import zipfile
import tempfile
import os
from pyspark.sql.functions import collect_set,collect_list, struct, first, when, coalesce,date_format
from pyspark.sql.functions import to_date, col, first, coalesce, when, collect_set, struct
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, collect_list, struct, first, when, coalesce, date_format, to_date, array_max
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType
import zipfile
import tempfile
import os
from math import radians, sin, cos, sqrt, atan2

# Initialize Spark session with more memory and executor settings
spark = SparkSession.builder \
    .appName("AIS Data Processing") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/ais_training_data.ais_data") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.ais_data") \
    .config("spark.mongodb.output.batchSize", "10000") \
    .config("spark.mongodb.output.bulk.enabled", "true") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()



24/10/09 20:53:13 WARN Utils: Your hostname, talal resolves to a loopback address: 127.0.1.1; using 192.168.10.112 instead (on interface wlo1)
24/10/09 20:53:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/talal/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/talal/.ivy2/cache
The jars for the packages stored in: /home/talal/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c64858f6-b94f-4c22-bf80-4f48a2010705;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 144ms :: artifacts dl 7ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   

In [2]:

# Define schema for the CSV files
schema = StructType([
    StructField("# Timestamp", StringType(), True),
    StructField("Type of mobile", StringType(), True),
    StructField("MMSI", StringType(), True),
    StructField("Latitude", DoubleType(), True),
    StructField("Longitude", DoubleType(), True),
    StructField("Navigational status", StringType(), True),
    StructField("ROT", StringType(), True),
    StructField("SOG", StringType(), True),
    StructField("COG", StringType(), True),
    StructField("Heading", StringType(), True),
    StructField("IMO", StringType(), True),
    StructField("Callsign", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Ship type", StringType(), True),
    StructField("Cargo type", StringType(), True),
    StructField("Width", StringType(), True),
    StructField("Length", StringType(), True),
    StructField("Type of position fixing device", StringType(), True),
    StructField("Draught", StringType(), True),
    StructField("Destination", StringType(), True),
    StructField("ETA", StringType(), True),
    StructField("Data source type", StringType(), True),
    StructField("A", StringType(), True),
    StructField("B", StringType(), True),
    StructField("C", StringType(), True),
    StructField("D", StringType(), True)
])

# Function to calculate Haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    return R * c

# Register UDF for Haversine distance
haversine_udf = udf(haversine_distance, DoubleType())

# Read port data from CSV
ports_df = spark.read.csv("ports_output.csv", header=True, inferSchema=True)
ports_df = ports_df.select("Port Name", "Latitude", "Longitude")

# Broadcast the ports dataframe
ports_broadcast = spark.sparkContext.broadcast(ports_df.collect())

# Function to find nearest port
def find_nearest_port(lat, lon):
    min_distance = float('inf')
    nearest_port = None
    for port in ports_broadcast.value:
        distance = haversine_distance(lat, lon, port['Latitude'], port['Longitude'])
        if distance < min_distance:
            min_distance = distance
            nearest_port = port['Port Name']
    return nearest_port

# Register UDF for finding nearest port
find_nearest_port_udf = udf(find_nearest_port, StringType())

def get_meaningful_value(column_name):
    return coalesce(
        first(when((col(column_name) != "Unknown value") & 
                   (col(column_name) != "Undefined") & 
                   (col(column_name).isNotNull()), 
                   col(column_name))),
        first(col(column_name))
    )

# Function to process each DataFrame
def process_dataframe(df):
    # Transform the data
    transformed_df = df.select(
        date_format(to_date(col("# Timestamp"), "dd/MM/yyyy"), "yyyy-MM-dd").alias("Date"),
        col("MMSI"),
        col("Latitude"),
        col("Longitude"),
        col("Navigational status").alias("Nav_Status"),
        col("Ship type").alias("Ship_Type"),
        col("Destination")
    )

    # Group by MMSI, collecting unique lat-long pairs and meaningful values of other fields
    grouped_df = transformed_df.groupBy("MMSI") \
        .agg(
            collect_list(struct("Latitude", "Longitude")).alias("Locations"),
            get_meaningful_value("Date").alias("Date"),
            get_meaningful_value("Nav_Status").alias("Nav_Status"),
            get_meaningful_value("Ship_Type").alias("Ship_Type"),
            get_meaningful_value("Destination").alias("Destination")
        )

    # Process the dataframe to update destinations
    processed_df = grouped_df.withColumn(
        "LastLocation", 
        array_max("Locations")
    ).withColumn(
        "NearestPort",
        find_nearest_port_udf(col("LastLocation.Latitude"), col("LastLocation.Longitude"))
    ).withColumn(
        "Destination",
        col("NearestPort")  # Always use the NearestPort as the Destination
    ).drop("LastLocation", "NearestPort")

    return processed_df

# Function to read CSV data from a ZIP file
def read_csv_from_zip(zip_path):
    start_processing = False
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        for file_name in zip_file.namelist():
            print(f"Processing file: {file_name}")
            #if file_name == "aisdk-2023-06-10.csv":
            start_processing = True
            if start_processing and file_name.endswith('.csv'):
                with zip_file.open(file_name) as csv_file:
                    # Create a temporary file
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.csv') as temp_file:
                        temp_file.write(csv_file.read())
                        temp_file_path = temp_file.name
                    
                    try:
                        # Read CSV content using the temporary file path
                        df = spark.read.option("header", "true").schema(schema).csv(temp_file_path)
                        
                        # Process the DataFrame
                        processed_df = process_dataframe(df)
                        # Write to MongoDB
                        processed_df.write.format("mongo") \
                            .option("uri", "mongodb://localhost:27017/ais_training_data.ais_data") \
                            .mode("append") \
                            .option("bulkWrite", "true") \
                            .save()
                        # Clear cache to free up memory
                        spark.catalog.clearCache()
                        
                        print(f"Completed processing file: {file_name}")
                    finally:
                        print("removing temp file")
                        os.unlink(temp_file_path)

zip_file_paths = [
    r"/media/talal/125866715866540F/FYP/Data AIS/aisdk-2023-07.zip"
]

# Process the ZIP files
for zip_file_path in zip_file_paths:
    print(f"Processing ZIP file: {os.path.basename(zip_file_path)}")
    try:
        read_csv_from_zip(zip_file_path)
        print(f"Finished processing {os.path.basename(zip_file_path)}")
    except Exception as e:
        print(f"Error processing {os.path.basename(zip_file_path)}: {str(e)}")
    print("-----------------------------------")

print("All ZIP files processed")

# Stop Spark session
spark.stop()

Processing ZIP file: aisdk-2023-07.zip
Processing file: aisdk-2023-07-01.csv


24/10/09 20:53:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Completed processing file: aisdk-2023-07-01.csv
removing temp file
Processing file: aisdk-2023-07-02.csv


Completed processing file: aisdk-2023-07-02.csv
removing temp file
Processing file: aisdk-2023-07-03.csv


Completed processing file: aisdk-2023-07-03.csv
removing temp file
Processing file: aisdk-2023-07-04.csv


...

Completed processing file: aisdk-2023-07-27.csv
removing temp file
Processing file: aisdk-2023-07-28.csv


Completed processing file: aisdk-2023-07-28.csv
removing temp file
Processing file: aisdk-2023-07-29.csv


Completed processing file: aisdk-2023-07-29.csv
removing temp file
Processing file: aisdk-2023-07-30.csv


Completed processing file: aisdk-2023-07-30.csv
removing temp file
Processing file: aisdk-2023-07-31.csv


Completed processing file: aisdk-2023-07-31.csv
removing temp file
Finished processing aisdk-2023-07.zip
-----------------------------------
All ZIP files processed


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_timestamp, collect_set, struct, collect_list, first, when, coalesce, date_format, to_date, array_max, expr
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, BooleanType
import zipfile
import tempfile
import os
from math import radians, sin, cos, sqrt, atan2

# Initialize Spark session with more memory and executor settings
spark = SparkSession.builder \
    .appName("AIS Data Processing") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/ais_training_data.specific_mmsi_data") \
    .config("spark.mongodb.output.batchSize", "10000") \
    .config("spark.driver.memory", "8g") \
    .config("spark.mongodb.output.bulk.enabled", "true") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Define schema for the CSV files
schema = StructType([
    StructField("# Timestamp", StringType(), True),
    StructField("Type of mobile", StringType(), True),
    StructField("MMSI", StringType(), True),
    StructField("Latitude", DoubleType(), True),
    StructField("Longitude", DoubleType(), True),
    StructField("Navigational status", StringType(), True),
    StructField("ROT", StringType(), True),
    StructField("SOG", StringType(), True),
    StructField("COG", StringType(), True),
    StructField("Heading", StringType(), True),
    StructField("IMO", StringType(), True),
    StructField("Callsign", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Ship type", StringType(), True),
    StructField("Cargo type", StringType(), True),
    StructField("Width", StringType(), True),
    StructField("Length", StringType(), True),
    StructField("Type of position fixing device", StringType(), True),
    StructField("Draught", StringType(), True),
    StructField("Destination", StringType(), True),
    StructField("ETA", StringType(), True),
    StructField("Data source type", StringType(), True),
    StructField("A", StringType(), True),
    StructField("B", StringType(), True),
    StructField("C", StringType(), True),
    StructField("D", StringType(), True)
])

# Function to calculate Haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    return R * c

# Read port data from CSV
ports_df = spark.read.csv("ports_output.csv", header=True, inferSchema=True)
ports_df = ports_df.select("Port Name", "Latitude", "Longitude")

# Broadcast the ports dataframe
ports_broadcast = spark.sparkContext.broadcast(ports_df.collect())

# UDF to find nearest port
@udf(StructType([
    StructField("NearestPort", StringType()),
    StructField("DistanceToPort", DoubleType())
]))
def find_nearest_port(lat, lon):
    min_distance = float('inf')
    nearest_port = None
    for port in ports_broadcast.value:
        distance = haversine_distance(lat, lon, port['Latitude'], port['Longitude'])
        if distance < min_distance:
            min_distance = distance
            nearest_port = port['Port Name']
    return (nearest_port, min_distance)

def get_meaningful_value(column_name):
    return coalesce(
        first(when((col(column_name) != "Unknown value") & 
                   (col(column_name) != "Undefined") & 
                   (col(column_name).isNotNull()), 
                   col(column_name))),
        first(col(column_name))
    )

# Function to process each DataFrame
def process_dataframe(df):
    # Transform the data
    transformed_df = df.select(
        date_format(to_date(col("# Timestamp"), "dd/MM/yyyy"), "yyyy-MM-dd").alias("Date"),
        col("MMSI"),
        col("Latitude"),
        col("Longitude"),
        col("Navigational status").alias("Nav_Status"),
        col("Ship type").alias("Ship_Type"),
        col("Destination")
    )

    # Group by MMSI, collecting unique lat-long pairs and meaningful values of other fields
    grouped_df = transformed_df.groupBy("MMSI") \
        .agg(
            collect_list(struct("Latitude", "Longitude")).alias("Locations"),
            get_meaningful_value("Date").alias("Date"),
            get_meaningful_value("Nav_Status").alias("Nav_Status"),
            get_meaningful_value("Ship_Type").alias("Ship_Type"),
            get_meaningful_value("Destination").alias("Destination")
        )
    # Process the dataframe to update destinations
    processed_df = grouped_df.withColumn(
        "LastLocation", 
        expr("Locations[size(Locations) - 1]")
    ).withColumn(
        "NearestPortInfo",
        find_nearest_port(col("LastLocation.Latitude"), col("LastLocation.Longitude"))
    ).withColumn(
        "NearestPort", 
        col("NearestPortInfo.NearestPort")
    ).withColumn(
        "DistanceToPort", 
        col("NearestPortInfo.DistanceToPort")
    ).withColumn(
        "IsPortStop", 
        when(col("DistanceToPort") < 10, True).otherwise(False)
    ).withColumn(
        "Destination",
        when(col("IsPortStop"), col("NearestPort")).otherwise(None)
    ).drop("LastLocation", "NearestPortInfo")
    return processed_df

# Function to read CSV data from a ZIP file
def read_csv_from_zip(zip_path):
    start_processing = False
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        for file_name in zip_file.namelist():
            print(f"Processing file: {file_name}")
            if file_name=="aisdk-2023-12-01.csv":
                start_processing = True
            if start_processing and file_name.endswith('.csv'):
                with zip_file.open(file_name) as csv_file:
                    # Create a temporary file
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.csv') as temp_file:
                        temp_file.write(csv_file.read())
                        temp_file_path = temp_file.name
                    
                    try:
                        # Read CSV content using the temporary file path
                        df = spark.read.option("header", "true").schema(schema).csv(temp_file_path)
                        
                        # Process the DataFrame
                        processed_df = process_dataframe(df)
                        processed_df.write.format("mongo") \
                            .option("uri", "mongodb://localhost:27017/ais_training_data.specific_mmsi_data") \
                            .mode("append") \
                            .option("bulkWrite", "true") \
                            .save()
                        # Clear cache to free up memory
                        spark.catalog.clearCache()
                        
                        print(f"Completed processing file: {file_name}")
                    finally:
                        print("removing temp file")
                        os.unlink(temp_file_path)

zip_file_paths = [
    r"/media/talal/125866715866540F/FYP/Data AIS/aisdk-2023-11.zip",
        r"/media/talal/125866715866540F/FYP/Data AIS/aisdk-2023-12.zip"
]

# Process the ZIP files
for zip_file_path in zip_file_paths:
    print(f"Processing ZIP file: {os.path.basename(zip_file_path)}")
    try:
        read_csv_from_zip(zip_file_path)
        print(f"Finished processing {os.path.basename(zip_file_path)}")
    except Exception as e:
        print(f"Error processing {os.path.basename(zip_file_path)}: {str(e)}")
    print("-----------------------------------")

print("All ZIP files processed")

# Stop Spark session
spark.stop()

24/10/11 15:48:17 WARN Utils: Your hostname, talal resolves to a loopback address: 127.0.1.1; using 192.168.10.112 instead (on interface wlo1)
24/10/11 15:48:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/talal/.ivy2/cache
The jars for the packages stored in: /home/talal/.ivy2/jars


:: loading settings :: url = jar:file:/home/talal/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5a6c5e27-db25-428a-8ef7-026cbcebeaba;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 137ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------------------------

Processing ZIP file: aisdk-2023-11.zip
Processing file: aisdk-2023-11-01.csv
Processing file: aisdk-2023-11-02.csv
Processing file: aisdk-2023-11-03.csv
Processing file: aisdk-2023-11-04.csv
Processing file: aisdk-2023-11-05.csv
Processing file: aisdk-2023-11-06.csv
Processing file: aisdk-2023-11-07.csv
Processing file: aisdk-2023-11-08.csv
Processing file: aisdk-2023-11-09.csv
Processing file: aisdk-2023-11-10.csv
Processing file: aisdk-2023-11-11.csv
Processing file: aisdk-2023-11-12.csv
Processing file: aisdk-2023-11-13.csv
Processing file: aisdk-2023-11-14.csv
Processing file: aisdk-2023-11-15.csv
Processing file: aisdk-2023-11-16.csv
Processing file: aisdk-2023-11-17.csv
Processing file: aisdk-2023-11-18.csv
Processing file: aisdk-2023-11-19.csv
Processing file: aisdk-2023-11-20.csv
Processing file: aisdk-2023-11-21.csv
Processing file: aisdk-2023-11-22.csv
Processing file: aisdk-2023-11-23.csv
Processing file: aisdk-2023-11-24.csv
Processing file: aisdk-2023-11-25.csv
Processing 

24/10/11 15:48:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


...

Completed processing file: aisdk-2023-12-27.csv
removing temp file
Processing file: aisdk-2023-12-28.csv


Completed processing file: aisdk-2023-12-28.csv
removing temp file
Processing file: aisdk-2023-12-29.csv


Completed processing file: aisdk-2023-12-29.csv
removing temp file
Processing file: aisdk-2023-12-30.csv


Completed processing file: aisdk-2023-12-30.csv
removing temp file
Processing file: aisdk-2023-12-31.csv


Completed processing file: aisdk-2023-12-31.csv
removing temp file
Finished processing aisdk-2023-12.zip
-----------------------------------
All ZIP files processed
